In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Import Data

In [2]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
display(train_df)
print('*'*40)
display(train_df.info())
display(test_df.info())

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


****************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


None

# Create function to help clean data

In [3]:
def clean_data(initial_train_df, initial_test_df, remove_columns=['Cabin'], encode_columns=['Sex'], encode=True, target=['Survived']):
    # Remove unwanted columns
    new_columns = list(initial_train_df.columns)
    # Create list of columns to keep in the training data
    for rcol in remove_columns:
        new_columns.remove(rcol)
    train_df_slice = initial_train_df[new_columns]
    # Test data doesn't have target columns so remove those too
    for targ in target:
        new_columns.remove(targ)
    test_df_slice = initial_test_df[new_columns]
    
    if encode:
        # One hot encode categorical values
        train_df_encode = pd.get_dummies(train_df_slice)
        test_df_encode = pd.get_dummies(test_df_slice)
        # train_df, test_df = train_df_encode.align(test_df_encode, join='left', axis=1)
    return train_df_encode.dropna(), test_df_encode.dropna()


train_df_cleaned, test_df_cleaned = clean_data(train_df, test_df)
display(train_df_cleaned.info())
display(train_df_cleaned)

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Columns: 1584 entries, PassengerId to Embarked_S
dtypes: bool(1577), float64(2), int64(5)
memory usage: 1.1 MB


None

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Ticket_W./C. 14263,Ticket_W./C. 6607,Ticket_W./C. 6608,Ticket_W./C. 6609,Ticket_W.E.P. 5734,Ticket_W/C 14208,Ticket_WE/P 5735,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,2,1,1,38.0,1,0,71.2833,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,1,3,26.0,0,0,7.9250,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,4,1,1,35.0,1,0,53.1000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,5,0,3,35.0,0,0,8.0500,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,39.0,0,5,29.1250,False,False,False,...,False,False,False,False,False,False,False,False,True,False
886,887,0,2,27.0,0,0,13.0000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
887,888,1,1,19.0,0,0,30.0000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
889,890,1,1,26.0,0,0,30.0000,False,False,False,...,False,False,False,False,False,False,False,True,False,False


# Trial with all available data besides the "Cabin" column

In [4]:
input_features = list(train_df_cleaned.columns)
input_features.remove('Survived')
X_train = train_df_cleaned[input_features]
y_train = train_df_cleaned['Survived']
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=0)

In [5]:
cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy')

array([0.70588235, 0.83193277, 0.79411765])

# Trial with more columns removed

In [6]:
train_df_cleaned, test_df_cleaned = clean_data(train_df, test_df, remove_columns=['Name', 'Ticket', 'Cabin', 'Embarked', 'Fare'])
input_features = list(train_df_cleaned.columns)
input_features.remove('Survived')
X_train = train_df_cleaned[input_features]
y_train = train_df_cleaned['Survived']
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=0)

In [7]:
cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy')

array([0.74789916, 0.7394958 , 0.83613445])

# Trial with PassengerId also removed

In [8]:
train_df_cleaned, test_df_cleaned = clean_data(train_df, test_df, remove_columns=['Name', 'Ticket', 'Cabin', 'Embarked'])
input_features = list(train_df_cleaned.columns)
input_features.remove('Survived')
input_features.remove('PassengerId')
X_train = train_df_cleaned[input_features]
y_train = train_df_cleaned['Survived']
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=0)

In [9]:
cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy')

array([0.79411765, 0.83193277, 0.8487395 ])

# Make predictions

In [10]:
test_df_cleaned
test_features = list(test_df_cleaned.columns)
test_features.remove('PassengerId')
X_test = test_df_cleaned[test_features]
predictions = clf.predict(X_test)

In [11]:
results = pd.DataFrame({'PassengerId': test_df_cleaned.PassengerId, 'Survived': predictions})
display(results)
results.to_csv('submission.csv', index=False)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
409,1301,1
411,1303,1
412,1304,0
414,1306,1
